In [ ]:
import requests
import csv
from datetime import datetime, timedelta, timezone
import time

# Tmap API 호출을 위한 URL 및 파라미터 설정
url = "https://apis.openapi.sk.com/tmap/traffic"
params = {
    "version": 1,
    "format": "json",
    "reqCoordType": "WGS84GEO",
    "resCoordType": "WGS84GEO",
    "zoomLevel": 12,
    "trafficType": "AUTO",
    "centerLon": 127.0016985,  # 서울 중심좌표
    "centerLat": 37.5642135
}

# API 호출 시 필요한 헤더 설정 (User-Agent 정보 추가)
headers = {
    "appKey": "", # T-map api key
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Mobile Safari/537.36",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Origin": "https://tmapapi.tmapmobility.com"
}

# 북위, 남위, 서경, 동경 경계값 설정
NORTH_LAT = 37.715133
SOUTH_LAT = 37.413294
WEST_LON = 126.734086
EAST_LON = 127.269311

# 좌표가 범위 내에 있는지 확인하는 함수
def is_within_boundary(coordinates):
    for lon, lat in coordinates:
        if not (SOUTH_LAT <= lat <= NORTH_LAT and WEST_LON <= lon <= EAST_LON):
            return False
    return True

# 데이터 수집 및 CSV 저장 함수
def collect_and_save_traffic_data():
    # GET 요청을 통해 교통 정보를 요청
    response = requests.get(url, params=params, headers=headers)

    # 응답 상태 코드가 200(성공)인지 확인
    if response.status_code == 200:
        traffic_data = response.json()  # JSON 형식으로 응답 데이터 파싱

        # CSV 파일 저장을 위한 열 제목 설정
        csv_columns = ['index', 'id', 'name', 'congestion', 'distance', 'updateTime', 'speed', 'direction', 'coordinates']

        # 결과 데이터에서 features를 추출
        features = traffic_data.get('features', [])

        # updateTime을 첫 번째 행에서 가져오기
        first_update_time = None

        # CSV 파일 생성 및 데이터 쓰기
        if features:
            # 첫 번째 행의 updateTime 저장
            first_update_time = features[0].get('properties', {}).get('updateTime', '')

            # 현재 KST 시간을 문자열로 변환하여 파일명에 사용
            current_time_kst = datetime.now().astimezone(timezone(timedelta(hours=9)))  # KST는 UTC+9
            filename = f"traffic_data_{current_time_kst.strftime('%Y%m%d_%H%M')}.csv"

            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                writer.writeheader()

                for idx, feature in enumerate(features):
                    properties = feature.get('properties', {})
                    geometry = feature.get('geometry', {})
                    coordinates = geometry.get('coordinates', [])

                    # id가 존재하지 않거나 좌표가 범위를 벗어나는 경우 제외
                    if properties.get('id') and is_within_boundary(coordinates):
                        # 위도, 경도 좌표를 문자열로 변환하여 저장
                        coord_str = '; '.join([f"({lon}, {lat})" for lon, lat in coordinates])

                        writer.writerow({
                            'index': properties.get('index', ''),
                            'id': properties.get('id', ''),
                            'name': properties.get('name', ''),
                            'congestion': properties.get('congestion', ''),
                            'distance': properties.get('distance', ''),
                            'updateTime': first_update_time,  # 모든 행에 동일한 updateTime 저장
                            'speed': properties.get('speed', ''),
                            'direction': properties.get('direction', ''),
                            'coordinates': coord_str  # 좌표 저장
                        })

            print(f"좌표가 포함된 CSV 파일이 '{filename}'로 성공적으로 저장되었습니다.")
        else:
            print("교통 정보가 없습니다.")
    else:
        print(f"Error: {response.status_code}")  # 오류 코드 출력

# 10분 간격으로 데이터 수집
while True:
    collect_and_save_traffic_data()
    time.sleep(600)  # 10분(600초) 대기
